In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df_gold = pd.read_excel('gold.xlsx')
df_covid = pd.read_csv('covid_data.csv')
#Clean Up of Gold Data frame
#Datetime -> Date Clean Up
df_gold['Date'] = pd.to_datetime(df_gold['Date']).dt.date
#df_gold['Date'] = pd.to_datetime(df_gold.Date, format='%Y-%m-%d')

#convert date column into datetime object
#df_gold['Date'] = df_gold['Date'].astype('datetime64[ns]')

#Remove non-close related tabs
df_gold.drop(['Volume','Open','High','Low'], axis=1, inplace=True)
#Rename close* -> Gold End Of Day Price
df_gold.rename(columns = {'Close/Last':'Close'}, inplace = True)
df_gold


,Date,Close
0,2021-11-19,1854.3
1,2021-11-18,1864.0
2,2021-11-17,1872.8
3,2021-11-16,1854.1
4,2021-11-15,1866.6
5,2021-12-11,1868.5
6,2021-11-11,1863.9
7,2021-10-11,1848.3
8,2021-09-11,1830.8
9,2021-08-11,1828.0


In [19]:
#Sync gold mon-fri Dates to weekly
# Using ISO 8601 definition for week 01 is the week with the first Thursday of the Gregorian year (i.e. of January)
#print(df_gold['Date'].dt.week)
#print(df_gold['Date'])

date_range = pd.period_range(start = '2019-12-30', end =  
                             '2022-01-02', freq = 'W-SUN')
date_range = date_range.map(str)
date_range = date_range.str.split('/').str[0]
date_range = pd.Series(date_range)
date_range

#gold_groupby = new_cases_daily_total.groupby([[i//7 for i in range(0,378)]],axis = 1).sum().T

#gold_weekly_avg = 
#new_cases_weekly_total_method_4 = new_cases_groupby_total_method_4.assign(Weeks = date_range)

0      2019-12-30
1      2020-01-06
2      2020-01-13
3      2020-01-20
4      2020-01-27
          ...    
100    2021-11-29
101    2021-12-06
102    2021-12-13
103    2021-12-20
104    2021-12-27
Length: 105, dtype: object